<a href="https://colab.research.google.com/github/RuolinZheng08/cmsc25460-optimization/blob/master/Homework2_lp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 2 

[CVXPY](https://www.cvxpy.org/tutorial/intro/index.html) is a Python-embedded modeling language for convex optimization problems. It allows you to express your problem in a natural way that follows the math, rather than in the restrictive standard form required by solvers.

# Exercise

### Shortest s-t path

Use CVXPY to construct and solve the linear programming formulation of $s$-$t$ shortest path from Problem 5 over the graph below, where edges are labeled by their length.

![Flow Graph](flow.png)

Submit your code and your solution (including primal and dual optimal solutions) as a pdf through gradescope with the rest of the homework.

__Note__: There are two strategies to accomplish this. The simplest one is to explicitly write all the constraints for the linear program algebraically as done in the examples you have seen before. If you are more comfortable with Python, you could try to use the object oriented approach described [here](https://www.cvxpy.org/examples/applications/OOCO.html).

In [0]:
import cvxpy as cp
import numpy as np

nodedata = ['S', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T']
nodedict = {elm: i for i, elm in enumerate(nodedata)}

edgedata = [   # (u, v, w)
    ("S", "A", 6),
    ("S", "B", 1),
    ("S", "C", 10),
    ("A", "B", 2),
    ("A", "D", 4),
    ("A", "E", 1),
    ("B", "E", 20),
    ("C", "B", 2),
    ("C", "F", 5),
    ("D", "E", 2),
    ("D", "G", 5),
    ("E", "F", 6),
    ("E", "G", 10),
    ("F", "T", 4),
    ("G", "T", 12)
]

num_nodes = len(nodedata)
num_edges = len(edgedata)

In [0]:
c = np.zeros(num_edges)
A = np.zeros((num_nodes, num_edges))

b = np.zeros(num_nodes)
# fill in -1 for S and 1 for T in b
b[0] = -1
b[-1] = 1

In [0]:
for e_idx, tup in enumerate(edgedata):
  out_node, in_node, length = tup
  c[e_idx] = length
  in_idx, out_idx = nodedict[in_node], nodedict[out_node]
  A[in_idx, e_idx] = 1
  A[out_idx, e_idx] = -1

In [114]:
# primal
x = cp.Variable(num_edges)
objective = cp.Minimize(c @ x)
constraints = [A @ x == b, x >= 0]
prob = cp.Problem(objective, constraints)
print(prob.solve())
print(x.value)

17.000000002057455
[ 9.99999999e-01  2.50164016e-10  1.20741352e-09 -5.56221834e-11
  2.48206727e-10  9.99999998e-01 -1.39941451e-11 -2.08542964e-10
  1.41809512e-09  1.23106097e-10  1.25045228e-10  9.99999998e-01
  1.93565816e-11  1.00000000e+00  1.43714559e-10]


In [117]:
# dual
y = cp.Variable(num_nodes)
objective = cp.Maximize(b @ y)
constraints = [A.T @ y <= c, y >= 0]
prob = cp.Problem(objective, constraints)
print(prob.solve())
print(y.value)

16.99999999824151
[ 1.22899124  7.22899124  1.07679878 10.34602567  8.85998402  8.22899124
 14.22899124 10.611074   18.22899124]


In [123]:
# recover the path
for i, val in enumerate(x.value):
  if np.isclose(val, 1):
    print(edgedata[i])

('S', 'A', 6)
('A', 'E', 1)
('E', 'F', 6)
('F', 'T', 4)


## Conclusion
The shortest path `S->A->E->F->T` has length 17.